In [1]:
from datetime import date
import math
import pandas as pd
from prophet import Prophet
from scipy.interpolate import interp1d
import numpy as np

# 1. Load and train Prophet model on your Nat_Gas.csv
df = pd.read_csv("Nat_Gas.csv")
df['ds'] = pd.to_datetime(df['Dates'])
df = df[['ds', 'Prices']].rename(columns={'Prices': 'y'})

model = Prophet()
model.fit(df)

# 2. Forecast 12+ months ahead
future = model.make_future_dataframe(periods=18, freq='M')  # extend as needed
forecast = model.predict(future)
forecast_df = forecast[['ds', 'yhat']].rename(columns={'ds': 'Date', 'yhat': 'Price'})
forecast_df.set_index('Date', inplace=True)

# 3. Create a price lookup function from Prophet forecast
price_interpolator = interp1d(
    forecast_df.index.astype(int),
    forecast_df['Price'].values,
    kind='linear',
    fill_value='extrapolate'
)

def get_forecast_price(dt):
    return float(price_interpolator(np.datetime64(dt).astype(int)))

# 4. Refactored contract pricing function using forecasted prices
def price_forecasted_contract(
    in_dates, out_dates, rate,
    storage_cost_rate, total_vol, injection_withdrawal_cost_rate
):
    volume = 0
    buy_cost = 0
    cash_in = 0
    all_dates = sorted(set(in_dates + out_dates))

    for current_date in all_dates:
        if current_date in in_dates:
            if volume <= total_vol - rate:
                volume += rate
                price = get_forecast_price(current_date)
                buy_cost += rate * price + rate * injection_withdrawal_cost_rate
                print(f'📦 Injected gas on {current_date} at forecasted price ${price:.2f}')
            else:
                print(f'⚠️ Cannot inject on {current_date} — storage full.')

        elif current_date in out_dates:
            if volume >= rate:
                volume -= rate
                price = get_forecast_price(current_date)
                cash_in += rate * price - rate * injection_withdrawal_cost_rate
                print(f'💸 Withdrawn gas on {current_date} at forecasted price ${price:.2f}')
            else:
                print(f'⚠️ Cannot withdraw on {current_date} — not enough stored gas.')

    storage_months = math.ceil((max(out_dates) - min(in_dates)).days / 30)
    storage_cost = storage_months * storage_cost_rate
    net_value = cash_in - buy_cost - storage_cost

    print("\n📊 Contract Summary")
    print(f"Storage months: {storage_months}")
    print(f"Total injected: {rate * len(in_dates):,} MMBtu")
    print(f"Total withdrawn: {rate * len(out_dates):,} MMBtu")
    print(f"Buy cost: ${buy_cost:,.2f}")
    print(f"Sell revenue: ${cash_in:,.2f}")
    print(f"Storage cost: ${storage_cost:,.2f}")
    print(f"✅ Net Contract Value: ${net_value:,.2f}")

    return net_value


c:\Users\user\Desktop\QuantitativeResearch\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
C:\Users\user\AppData\Local\Temp\ipykernel_15744\2016824133.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['ds'] = pd.to_datetime(df['Dates'])
19:03:58 - cmdstanpy - INFO - Chain [1] start processing
19:03:58 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\user\Desktop\QuantitativeResearch\venv\Lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [2]:
from datetime import date

injection_dates = [date(2025, 6, 30), date(2025, 7, 31)]
withdrawal_dates = [date(2026, 1, 31), date(2026, 2, 28)]

result = price_forecasted_contract(
    in_dates=injection_dates,
    out_dates=withdrawal_dates,
    rate=1_000_000,
    storage_cost_rate=80_000,
    total_vol=2_000_000,
    injection_withdrawal_cost_rate=0.01  # $0.01 per MMBtu
)


📦 Injected gas on 2025-06-30 at forecasted price $-330.98
📦 Injected gas on 2025-07-31 at forecasted price $-330.98
💸 Withdrawn gas on 2026-01-31 at forecasted price $-330.98
💸 Withdrawn gas on 2026-02-28 at forecasted price $-330.98

📊 Contract Summary
Storage months: 9
Total injected: 2,000,000 MMBtu
Total withdrawn: 2,000,000 MMBtu
Buy cost: $-661,941,393.03
Sell revenue: $-661,981,393.03
Storage cost: $720,000.00
✅ Net Contract Value: $-760,000.00
